As a Data Scientist, I was contacted by a friend of mine who wanted some advice on where to open a Mexican restaurant in Washington D.C. <br>
In order to help him I will be using Foursquare data, aqcuired by their API, to determine the best place to his new endeavor.

In [26]:
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [17]:
CLIENT_ID = '#############################'
CLIENT_SECRET = '#############################' 
ACCESS_TOKEN = '#############################'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Retrieving the city coordinates

In [18]:
address = 'Washington DC, CO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.8949924 -77.0365581


Using Foursquare API to filter restaurants around 1 km of the city center

In [27]:
search_query = 'Restaurant'
radius = 1000

In [20]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6074953b5352b65ae226fc2c'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4bf1ee8da09076b0d7f329d4',
    'name': '1331 Restaurant & Lounge',
    'location': {'address': '1331 Pennsylvania Avenue NW',
     'crossStreet': 'btwn 13th & 14th St NW',
     'lat': 38.8963301,
     'lng': -77.0310183,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.8963301,
       'lng': -77.0310183}],
     'distance': 502,
     'postalCode': '20004',
     'cc': 'US',
     'city': 'Washington',
     'state': 'D.C.',
     'country': 'United States',
     'formattedAddress': ['1331 Pennsylvania Avenue NW (btwn 13th & 14th St NW)',
      'Washington, D.C. 20004',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/im

In [22]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-22-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4bf1ee8da09076b0d7f329d4,1331 Restaurant & Lounge,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1618253116,False,1331 Pennsylvania Avenue NW,btwn 13th & 14th St NW,38.896330,-77.031018,"[{'label': 'display', 'lat': 38.8963301, 'lng'...",...,D.C.,United States,[1331 Pennsylvania Avenue NW (btwn 13th & 14th...,1493234,https://www.grubhub.com/restaurant/1331-bar--l...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
1,45fec67bf964a5208c441fe3,Harry's Restaurant & Saloon,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1618253116,False,436 11th St NW,at E St. NW,38.895911,-77.027496,"[{'label': 'display', 'lat': 38.89591071893386...",...,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40b13b00f964a52045f31ee3,Stan's Restaurant & Lounge,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1618253116,False,1029 Vermont Ave NW,NaN,38.903516,-77.032997,"[{'label': 'display', 'lat': 38.903516, 'lng':...",...,D.C.,United States,"[1029 Vermont Ave NW, Washington, D.C. 20005, ...",1253491,https://www.grubhub.com/restaurant/stans-resta...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
3,4bc4d92f4cdfc9b6a3409921,Chalin's Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618253116,False,1912 I St NW,btwn 19th & 20th Sts NW,38.901336,-77.044103,"[{'label': 'display', 'lat': 38.90133602105807...",...,D.C.,United States,"[1912 I St NW (btwn 19th & 20th Sts NW), Washi...",68721,https://www.grubhub.com/restaurant/chalins-191...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
4,4bd181be77b29c74cda88c82,Harriet's Family Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1618253116,False,436 11th St NW,at E St. NW,38.895910,-77.027042,"[{'label': 'display', 'lat': 38.89590976442353...",...,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",1756674,https://www.grubhub.com/restaurant/harriets-fa...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN


In [23]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,1331 Restaurant & Lounge,American Restaurant,1331 Pennsylvania Avenue NW,btwn 13th & 14th St NW,38.896330,-77.031018,"[{'label': 'display', 'lat': 38.8963301, 'lng'...",502,20004,US,Washington,D.C.,United States,[1331 Pennsylvania Avenue NW (btwn 13th & 14th...,4bf1ee8da09076b0d7f329d4
1,Harry's Restaurant & Saloon,Bar,436 11th St NW,at E St. NW,38.895911,-77.027496,"[{'label': 'display', 'lat': 38.89591071893386...",791,20004,US,Washington,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",45fec67bf964a5208c441fe3
2,Stan's Restaurant & Lounge,American Restaurant,1029 Vermont Ave NW,NaN,38.903516,-77.032997,"[{'label': 'display', 'lat': 38.903516, 'lng':...",997,20005,US,Washington,D.C.,United States,"[1029 Vermont Ave NW, Washington, D.C. 20005, ...",40b13b00f964a52045f31ee3
3,Chalin's Restaurant,Chinese Restaurant,1912 I St NW,btwn 19th & 20th Sts NW,38.901336,-77.044103,"[{'label': 'display', 'lat': 38.90133602105807...",962,20006,US,Washington,D.C.,United States,"[1912 I St NW (btwn 19th & 20th Sts NW), Washi...",4bc4d92f4cdfc9b6a3409921
4,Harriet's Family Restaurant,Breakfast Spot,436 11th St NW,at E St. NW,38.895910,-77.027042,"[{'label': 'display', 'lat': 38.89590976442353...",830,20004,US,Washington,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",4bd181be77b29c74cda88c82
5,BAO Restaurant & Lounge,Chinese Restaurant,1018 Vermont Ave NW,NaN,38.903255,-77.033806,"[{'label': 'display', 'lat': 38.90325546264648...",950,20005,US,Washington,D.C.,United States,"[1018 Vermont Ave NW, Washington, D.C. 20005, ...",4b213d32f964a520c83824e3
6,Restaurant Association of Metropolitan Washing...,None,NaN,NaN,38.903150,-77.037924,"[{'label': 'display', 'lat': 38.90314963504873...",915,20036,US,Washington,D.C.,United States,"[Washington, D.C. 20036, United States]",4c8e69244e95236ae8c57bca
7,Lafayette Restaurant,American Restaurant,"800 16th St., N.W.",H Street,38.900145,-77.036826,"[{'label': 'display', 'lat': 38.90014471111636...",574,20006,US,Washington,D.C.,United States,"[800 16th St., N.W. (H Street), Washington, D....",4e4e534cbd4101d0d7a7c47c
8,Restaurant Association Metropolitan Washington,None,NaN,NaN,38.905775,-77.038667,"[{'label': 'display', 'lat': 38.905775, 'lng':...",1214,NaN,US,Washington,D.C.,United States,"[Washington, D.C., United States]",4da457b1bf22a143d963dbf8
9,Willard Room Restaurant,American Restaurant,1401 Pennsylvania Ave NW,NaN,38.896461,-77.032463,"[{'label': 'display', 'lat': 38.89646148681640...",390,20004,US,Washington,D.C.,United States,"[1401 Pennsylvania Ave NW, Washington, D.C. 20...",4bad788ff964a52026543be3


Creating a map the restaurants found in Foursquare data

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map